# Einführung Pandas

**Inhalt:** Pandas kennenlernen

**Nötige skills**: Keine

**Lerniziele**:
- Eine erste Idee erhalten: Wozu ist Pandas gut?

### Was ist Pandas?
Kurz gesagt: Pandas ist Excel für Python.

Das Modul ermöglicht ähnliche Operationen wie ein Tabellenkalkulationsprogramm, aber
- in Logbuchform statt als Tabellenblatt
- mit reproduzierbarem Code
- viel mehr Funktionalität
- kombinierbar mit anderen Modulen
- mit der Funktion, Daten zu exportieren

Ohne Pandas (fast) kein Datenjournalismus mit Python.

### Installation

Anaconda packages:
- pandas
- matplotlib

In [ ]:
import pandas as pd

In [ ]:
print ("Hooray!")

# Das Beispiel

**Corona-Fälle**: Die Johns Hopkins University hat eine globale Datenbank aufgebaut: https://coronavirus.jhu.edu/map.html

Die Daten dazu sind auf Github abgelegt: https://github.com/CSSEGISandData/COVID-19/

Wir schauen uns die Daten an, lernen ein paar Analysemöglichkeiten kennen und bauen die Charts für folgende Story nach: https://www.republik.ch/2020/07/20/in-schwellenlaendern-beginnt-die-covid-krise-gerade-erst-so-richtig

## Daten laden

Pandas kann lokale Datenquellen wie Excelfiles oder csv-Dateien laden. Praktisch: Man kann die Dateien auch direkt aus dem Netz holen.

Wir holen uns zwei verschiedene Datensets:
- eines mit den Fallzahlen
- eines mit den Todeszahlen

Vorhanden sind diese Länder jeweils für Länder und ihre Provinzen. Die Zahlen sind **kumuliert über die Zeit** angegeben.

**Fallzahlen**

In [ ]:
path = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

In [ ]:
df_cases = pd.read_csv(path)

In [ ]:
df_cases.head()

**Todeszahlen**

In [ ]:
path = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

In [ ]:
df_deaths = pd.read_csv(path)

In [ ]:
df_deaths.head()

### Mögliche Fragestellungen

Bevor wir mit dem Beispiel beginnen, lasst uns kurz nachdenken:
- Was könnten interessante Fragestellungen sein, die sich mit diesem Datensatz beantworten lassen?
- Was für interessante Analysen könnten wir mit den Daten machen?
- An was für Grafiken könnten wir interessiert sein?

In [ ]:
# Platz zum Antworten aufschreiben

### Unsere Fragestellung

In diesem Notebook schauen wir uns zwei Anwendungen an:
1. Inzidenzkurve für verschiedene Länder: Welche Dynamik hat die Epidemie in der Schweiz, in den USA, in Finnland, etc.?
1. Verlauf der Epidemie nach Ländergruppen: An welchem Punkt stehen die Industrieländer, Schwellenländer, Entwicklungsländer?

## Daten arrangieren

Bevor wir mit den Daten arbeiten können, müssen wir sie noch etwas arrangieren.

**Fallzahlen**

Zuerst schmeissen wir ein paar überflüssige Angaben raus, die uns nicht interessieren...

In [ ]:
df_cases = df_cases.drop(columns=['Province/State', 'Lat', 'Long'])

Danach gruppieren wir die Daten nach Land und geben den Datumsangaben ein richtiges Format...

In [ ]:
df_cases = df_cases.groupby('Country/Region').sum().T
df_cases.index = pd.to_datetime(df_cases.index, format='%m/%d/%y')

Und schliesslich bilden wir die täglichen Differenzen...

In [ ]:
df_cases = df_cases.diff()[1:]
df_cases.index.name = 'date'
df_cases = df_cases.astype(int)

Nun haben wir ein "sauberes" Datenset:

In [ ]:
df_cases.head()

In [ ]:
df_cases.tail()

Zur Vorbereitung für später kreieren wir gleich noch zwei weitere Datensets:
- eines mit dem gleitenden Mittelwert über sieben Tage hinweg
- eines mit dem wöchentlichen Mittelwert (das ist nicht dasselbe!)

In [ ]:
df_cases_r = df_cases.rolling(7).mean()

In [ ]:
df_cases_w = df_cases.resample('W').sum()

**Todesfälle**

Mit den Todesfällen machen wir genau dasselbe

In [ ]:
df_deaths = df_deaths.drop(columns=['Province/State', 'Lat', 'Long'])
df_deaths = df_deaths.groupby('Country/Region').sum().T
df_deaths.index = pd.to_datetime(df_deaths.index, format='%m/%d/%y')

In [ ]:
df_deaths = df_deaths.diff()[1:]
df_deaths.index.name = 'date'
df_deaths = df_deaths.astype(int)

In [ ]:
df_deaths_r = df_deaths.rolling(7).mean()

In [ ]:
df_deaths_w = df_deaths.resample('W').sum()

## 1. Verlauf in einzelnen Ländern

Pandas ist ein praktisches Tool, um explorative Datenanalysen durchzuführen - und logbuchartig festzuhalten, was dabei rauskommt.

Wir können zum Beispiel mal die Covid-Fallzahlen-Kurve in der Schweiz rausziehen und diese mit einer einfachen Codezeile auch gleich plotten:

In [ ]:
df_cases['Switzerland']

In [ ]:
df_cases['Switzerland'].plot(kind='bar')

### Schöne Plots

Leider sehen die Standardplots manchmal etwas hässlich aus.

Wir können - sofern wir Zeit haben - uns den Plot mit ein paar Codezeilen etwas verschönern.

Dazu brauchen wir zuerst noch ein paar weitere Bibliotheken.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

Es gibt unzählige Sachen, die man an einem Chart einstellen kann, bis er "schön" ist:
- Einen aussagekräftigen Titel
- Die Achsenbeschriftungen und Labels ("Ticks")
- mehrere Charts nebeneinander
- mehrere Elemente in einem Chart (zB Trendlinien)
- ...

In [ ]:
country = 'Switzerland'
color='red'

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,10))

data = df_cases
ax1.bar(data.index, data[country], color=color)
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax1.set_title('Case incidence in ' + country)
ax1.set_ylabel('Count')
ax1.set_xlabel('Date')

data = df_cases.rolling(7).mean()
ax1.plot(data.index, data[country], color='black')

data = df_deaths
ax2.bar(data.index, data[country], color=color)
ax2.xaxis.set_major_locator(mdates.MonthLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax2.set_title('Deaths in ' + country)
ax2.set_ylabel('Count')
ax2.set_xlabel('Date')

data = df_deaths.rolling(7).mean()
ax2.plot(data.index, data[country], color='black')

Damit wir diesen Code nicht jedesmal neu schreiben müssen, bauen wir uns eine Funktion dafür. Wir nennen Sie `countrychart()`. Sie erhält:
- das Land, dessen Kurven wir plotten wollen
- die Farbe

In [ ]:
def countrychart(country, color):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,4))

    data = df_cases
    ax1.bar(data.index, data[country], color=color)
    ax1.xaxis.set_major_locator(mdates.MonthLocator())
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax1.set_title('Case incidence in ' + country)
    ax1.set_ylabel('Count')
    ax1.set_xlabel('Date')
    
    data = df_cases.rolling(7).mean()
    ax1.plot(data.index, data[country], color='black')

    data = df_deaths
    ax2.bar(data.index, data[country], color=color)
    ax2.xaxis.set_major_locator(mdates.MonthLocator())
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax2.set_title('Deaths in ' + country)
    ax2.set_ylabel('Count')
    ax2.set_xlabel('Date')
    
    data = df_deaths.rolling(7).mean()
    ax2.plot(data.index, data[country], color='black')

### ein paar Länder...

In [ ]:
countrychart('Switzerland', 'tab:red')

In [ ]:
countrychart('Sweden', 'orange')

In [ ]:
countrychart('US', 'royalblue')

In [ ]:
countrychart('Brazil', 'forestgreen')

... etc. Sie können hier weitere Länder ausprobieren.

Eine Liste der Farbnamen gibt es übrigens hier: https://matplotlib.org/3.1.0/gallery/color/named_colors.html

## 2. Ländergruppen

Hier sind die Schritte, mit denen die Charts zur Schwellenländer-Story erstellt wurden.

Wir interessieren uns dabei für das *big picture*, wollen drei Ländergruppen bilden:
- Industrieländer (reiche Länder)
- Schwellenländer (aufsteigende Wirtschaften)
- Entwicklungsländer (arme Länder)

Dazu brauchen wir aber noch eine Zusatzinformation: Welches Land ist in welcher Gruppe?

### Weitere Daten laden

Eine Liste dazu haben wir bereits vorbereitet :-)

In [ ]:
path = 'dataprojects/covid/countrylist_categories.xlsx'

In [ ]:
df_cat = pd.read_excel(path)

In [ ]:
df_cat.head()

### Daten kategorisieren und summieren

Zuerst erstellen wir drei Listen mit allen Ländernamen, die jeweils in eine Kategorie fallen:

In [ ]:
cat_E = df_cat[df_cat['type'] == 'E']['Country/Region']
cat_S = df_cat[df_cat['type'] == 'S']['Country/Region']
cat_I = df_cat[df_cat['type'] == 'I']['Country/Region']

In [ ]:
cat_E

Danach filtern wir die anhand dieser Liste, summieren sie und bilden drei neue Einträge:

In [ ]:
df_cases['Entwicklungsländer'] = df_cases[cat_E].sum(axis=1)
df_cases['Schwellenländer'] = df_cases[cat_S].sum(axis=1)
df_cases['Industrieländer'] = df_cases[cat_I].sum(axis=1)

In [ ]:
df_deaths['Entwicklungsländer'] = df_deaths[cat_E].sum(axis=1)
df_deaths['Schwellenländer'] = df_deaths[cat_S].sum(axis=1)
df_deaths['Industrieländer'] = df_deaths[cat_I].sum(axis=1)

### Plotten

In [ ]:
countrychart('Entwicklungsländer', '#e377c2')

In [ ]:
countrychart('Schwellenländer', '#17becf')

In [ ]:
countrychart('Industrieländer', '#bcbd22')

### Varianten ausprobieren

In unserem Onlinecharting-Tool wollen wir die Daten nicht auf Tages-, sondern auf Wochenbasis zeigen.

Weil inzwischen neue Länder dazugekommen sind, müssen wir die Kalkulation nochmals ausführen.

In [ ]:
df_cases_w = df_cases.resample('W').sum()
df_deaths_w = df_deaths.resample('W').sum()

Um das schön zu plotten, müssen wir die Funktion leicht anpassen:

In [ ]:
def countrychart_weekly(country, color):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,4))

    data = df_cases_w
    ax1.bar(data.index.strftime(date_format="%d %b %Y"), data[country], color=color)
    ax1.xaxis.set_major_locator(ticker.LinearLocator(numticks=9))
    ax1.set_title('Case incidence in ' + country)
    ax1.set_ylabel('Count')
    ax1.set_xlabel('Date')
    
    data = df_deaths_w
    ax2.bar(data.index.strftime(date_format="%d %b %Y"), data[country], color=color)
    ax2.xaxis.set_major_locator(ticker.LinearLocator(numticks=9))
    ax2.set_title('Deaths in ' + country)
    ax2.set_ylabel('Count')
    ax2.set_xlabel('Date')
    
    fig.autofmt_xdate(rotation=90)

In [ ]:
countrychart_weekly('Entwicklungsländer', '#e377c2')

In [ ]:
countrychart_weekly('Schwellenländer', '#17becf')

In [ ]:
countrychart_weekly('Industrieländer', '#bcbd22')

### Daten exportieren

Um unsere Charts ins Web-CMS zu übernehmen, müssen wir sie in brauchbarer Form exportieren.

Wir wollen drei verschiedene csv-Files haben (je eines pro Ländergruppe).

In [ ]:
countries = [
    'Entwicklungsländer',
    'Schwellenländer',
    'Industrieländer'
]

In [ ]:
path = 'dataprojects/covid/charts/'

In [ ]:
for country in countries:
    df_deaths_w[[country]].round(3).rename(columns={country: 'value'}).to_csv(path + country + '_Fallzahlen.csv')
    df_deaths_w[[country]].round(3).rename(columns={country: 'value'}).to_csv(path + country + '_Todeszahlen.csv')

## Resumé

Wir haben in diesem Notebook gesehen:
- Pandas ist ziemlich praktisch, um Daten direkt aus dem Web abzuzapfen und zu analysieren
- Man kann logbuchartig die Bearbeitung festhalten und dokumentieren
- Die Analyse lässt sich jederzeit wiederholen und updaten
- Es braucht relativ wenig Code, um Daten zu transformieren (zB wöchentliche Summen)
- Plotten in Pandas ist simpel, um ersten Eindruck zu erhalten
- Schöne Plots in Pandas sind aber mühsam (aber meistens auch nicht nötig)

Doch die Einfachheit entbindet uns nicht davon, zu überlegen, ob das, was wir tun, auch sinnvoll ist!

**Frage**: Was sind mögliche Probleme und Fallstricke, die wir bei den obigen Charts haben?

In [ ]:
# Platz für Antworten